In [1]:
# 导入必要的库
from sklearn import datasets
import import_ipynb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.tree import DecisionTreeClassifier
import torch
import torch.nn as nn
import numpy as np
import random
from sklearn.model_selection import StratifiedKFold, cross_val_predict
from sklearn.model_selection import cross_val_score
from sklearn.neural_network import MLPClassifier
from sklearn import metrics
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.impute import KNNImputer
import torch.nn.functional as F
from einops import rearrange, repeat, reduce
from utils import *

importing Jupyter notebook from utils.ipynb
importing Jupyter notebook from network.ipynb


In [2]:
seed=42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # 使用 GPU 

In [3]:
class GraphConvolution(nn.Module):
    def __init__(self, in_features, out_features, bias=True):
        super().__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.weight = nn.Parameter(torch.FloatTensor(in_features, out_features))
        if bias:
            self.bias = nn.Parameter(torch.FloatTensor(out_features))
        nn.init.xavier_normal_(self.weight.data)
        if self.bias is not None:
            self.bias.data.fill_(0.0)
    
    def forward(self, x, adj):
        support = torch.mm(x, self.weight)
        output = torch.sparse.mm(adj, support)
        if self.bias is not None:
            return output + self.bias
        else:
            return output

In [4]:
class GCN_E(nn.Module):
    def __init__(self, in_dim, hgcn_dim, dropout):
        super().__init__()
        self.gc1 = GraphConvolution(in_dim, hgcn_dim[0])
        self.gc2 = GraphConvolution(hgcn_dim[0], hgcn_dim[1])
        self.gc3 = GraphConvolution(hgcn_dim[1], hgcn_dim[2])
        self.dropout = dropout

    def forward(self, x, adj):
        x = self.gc1(x, adj)
        x = F.leaky_relu(x, 0.25)
        
        x = F.dropout(x, self.dropout, training=self.training)
        
        x = self.gc2(x, adj)
        x = F.leaky_relu(x, 0.25)
        
        x = F.dropout(x, self.dropout, training=self.training)

        x = self.gc3(x, adj)
        x = F.leaky_relu(x, 0.25)
        return x


In [5]:
def xavier_init(m):
    if type(m) == nn.Linear:
        nn.init.xavier_normal_(m.weight)
        if m.bias is not None:
           m.bias.data.fill_(0.0)

In [6]:
class Classifier_1(nn.Module):
    def __init__(self, in_dim, out_dim):
        super().__init__()
        self.clf = nn.Sequential(nn.Linear(in_dim, out_dim))
        self.clf.apply(xavier_init)

    def forward(self, x):
        x = self.clf(x)
        return x

In [7]:
class Graph(nn.Module):
    def __init__(self, in_dim, hgcn_dim, dropout, classifier_out_dim):
        super().__init__()
        self.gcn = GCN_E(in_dim, hgcn_dim, dropout)
#         self.classifier = Classifier_1(hgcn_dim[-1], classifier_out_dim)

    def forward(self, x, adj):
        x = self.gcn(x, adj)
#         x = self.classifier(x)
        return x


In [8]:
class VCDN(nn.Module):
    def __init__(self, num_view, num_cls, hvcdn_dim):
        super().__init__()
        self.num_cls = num_cls
        self.model = nn.Sequential(
            nn.Linear(pow(num_cls, num_view), hvcdn_dim),
            nn.LeakyReLU(0.25),
            nn.Linear(hvcdn_dim, num_cls)
        )
        self.model.apply(xavier_init)
        
    def forward(self, in_list):
        num_view = len(in_list)
        for i in range(num_view):
            in_list[i] = torch.sigmoid(in_list[i])
        x = torch.reshape(torch.matmul(in_list[0].unsqueeze(-1), in_list[1].unsqueeze(1)),(-1,pow(self.num_cls,2),1))
        for i in range(2,num_view):
            x = torch.reshape(torch.matmul(x, in_list[i].unsqueeze(1)),(-1,pow(self.num_cls,i+1),1))
        vcdn_feat = torch.reshape(x, (-1,pow(self.num_cls,num_view)))
        output = self.model(vcdn_feat)

        return output

In [9]:
class SubNet(nn.Module):
    def __init__(self, in_size, hidden_size):
        super(SubNet, self).__init__()
        encoder1 = nn.Sequential(nn.Linear(in_size, hidden_size),nn.Tanh())
        encoder2 = nn.Sequential(nn.Linear(hidden_size, hidden_size), nn.Tanh())
        self.encoder = nn.Sequential(encoder1, encoder2)
    def forward(self, x):
        y = self.encoder(x)
        return y

In [ ]:
import torch
import torch.nn as nn

class CMFM(nn.Module):
    def __init__(self, in_size, output_dim, dropout, dim_hvcdn=64):
        super(CMFM, self).__init__()
        
        self.lmf = LMF(in_size, 16)
    
        self.encoder = nn.Sequential(
            nn.Linear(in_size, 2), 
            nn.LeakyReLU(negative_slope=0.01),  # 用 LeakyReLU 替代 ReLU
            nn.Dropout(p=dropout)
        )

        ### Path
        self.linear_h1 = nn.Sequential(
            nn.Linear(in_size, in_size), 
            nn.LeakyReLU(negative_slope=0.01)  # 用 LeakyReLU 替代 ReLU
        )
        self.linear_z1 = nn.Bilinear(in_size, in_size * 2, in_size)
        self.linear_o1 = nn.Sequential(
            nn.Linear(in_size, in_size), 
            nn.LeakyReLU(negative_slope=0.01),  # 用 LeakyReLU 替代 ReLU
            nn.Dropout(p=dropout)
        )
        
        # 定义初始化函数
        self.apply(self.init_weights)
    
    def init_weights(self, m):
        if isinstance(m, nn.Linear):
            nn.init.xavier_normal_(m.weight)
            if m.bias is not None:
                m.bias.data.fill_(0.0)
        elif isinstance(m, nn.Bilinear):
            nn.init.xavier_normal_(m.weight)
            if m.bias is not None:
                m.bias.data.fill_(0.0)
    
    def forward(self, dec_logits1, dec_logits2, dec_logits3):
        h1 = self.linear_h1(dec_logits1)
        vec31 = torch.cat((dec_logits2, dec_logits3), dim=1)
        z1 = self.linear_z1(dec_logits1, vec31)
        o1 = self.linear_o1(torch.sigmoid(z1) * h1)
        
        h2 = self.linear_h1(dec_logits2)
        vec32 = torch.cat((dec_logits1, dec_logits3), dim=1)
        z2 = self.linear_z1(dec_logits2, vec32)
        o2 = self.linear_o1(torch.sigmoid(z2) * h2)
        
        h3 = self.linear_h1(dec_logits3)
        vec33 = torch.cat((dec_logits1, dec_logits2), dim=1)
        z3 = self.linear_z1(dec_logits2, vec33)
        o3 = self.linear_o1(torch.sigmoid(z3) * h3)
        
        lmf = self.lmf(o1, o2, o3)
        
        return lmf, dec_logits1, dec_logits2, dec_logits3

In [ ]:
class TCP(nn.Module):
    def __init__(self, in_dim, hidden_dim, num_class, dropout):
        super().__init__()
        self.views = len(in_dim)+1
        self.classes = num_class
        self.dropout = dropout

        self.TCPConfidenceLayer = nn.ModuleList([Classifier_1(hidden_dim[0], 1) for _ in range(self.views)])
        self.TCPClassifierLayer = nn.ModuleList([Classifier_1(hidden_dim[0], num_class) for _ in range(self.views)])
#         self.classifier = Classifier_1(hidden_dim, num_class)

        self.MMClasifier = []
        for layer in range(1, len(hidden_dim) - 1):
            self.MMClasifier.append(Classifier_1(self.views * hidden_dim[0], hidden_dim[layer]))
            self.MMClasifier.append(nn.ReLU())
            self.MMClasifier.append(nn.Dropout(p=dropout))
        if len(self.MMClasifier):
            self.MMClasifier.append(Classifier_1(hidden_dim[-1], num_class))
        else:
            self.MMClasifier.append(Classifier_1(self.views * hidden_dim[-1], num_class))
        
        self.MMClasifier = nn.Sequential(*self.MMClasifier)

    def forward(self, feature, label=None, infer=False):
        criterion = torch.nn.CrossEntropyLoss(reduction='none')
        TCPLogit, TCPConfidence =  dict(), dict()
        all_cord = []
        for view in range(self.views):
            feature[view] = F.relu(feature[view])
            feature[view] = F.dropout(feature[view], self.dropout, training=self.training)
            TCPLogit[view] = self.TCPClassifierLayer[view](feature[view])
            TCPConfidence[view] = self.TCPConfidenceLayer[view](feature[view])
            feature[view] = feature[view] * TCPConfidence[view]
            all_cord.append(feature[view])
        
        # 使用 torch.cat() 沿着第 1 维（列维度）拼接
        MMfeature = torch.cat(all_cord, dim=1)
        MMlogit = self.MMClasifier(MMfeature)
        if infer:
            return MMlogit
        MMLoss = torch.mean(criterion(MMlogit, label))
        for view in range(self.views):
            pred = F.softmax(TCPLogit[view], dim=1)
            p_target = torch.gather(input=pred, dim=1, index=label.unsqueeze(dim=1)).view(-1)
            confidence_loss = torch.mean(
                F.mse_loss(TCPConfidence[view].view(-1), p_target) + criterion(TCPLogit[view], label))
            MMLoss = MMLoss + confidence_loss
        return MMLoss, MMlogit, MMfeature
    # def get_embedding(self, feature):  # ← 👈 你添加在这里
    #     all_cord = []
    #     for view in range(self.views):
    #         f = F.relu(feature[view])
    #         f = F.dropout(f, self.dropout, training=False)
    #         confidence = self.TCPConfidenceLayer[view](f)
    #         f = f * confidence
    #         all_cord.append(f)
    #     MMfeature = torch.cat(all_cord, dim=1)
    #     return MMfeature

In [12]:
def exists(val):
    return val is not None

In [13]:
def default(val, default_val):
    return val if val is not None else default_val

In [14]:
def init_zero_(layer):
    nn.init.constant_(layer.weight, 0.)
    if exists(layer.bias):
        nn.init.constant_(layer.bias, 0.)

In [15]:
class GEGLU(nn.Module):
    def forward(self, x):
        x, gates = x.chunk(2, dim=-1)
        return x * F.gelu(gates)

In [16]:
class Embedding_layer(nn.Module):
    def __init__(self, feature_len, embeding_dim):
        super(Embedding_layer, self).__init__()
        self.M = torch.nn.Parameter(torch.tensor(scale(np.random.rand(feature_len, embeding_dim))).float(),
                                    requires_grad=True)

    def forward(self, enc_inputs):  # X: [batch_size, feature_len,embeding_len]
        if enc_inputs.shape[2]>2:
            X=enc_inputs[:,:,0:1]* self.M
            for i in range(1,enc_inputs.shape[2]):
                X = torch.cat((X,enc_inputs[:,:,i:(i+1)] * self.M),2)
        else:
            X = enc_inputs * self.M
        return X

In [17]:
class FeedForward(nn.Module):
    def __init__(
            self,
            dim,
            mult=4,
            dropout=0.
    ):
        super().__init__()
        self.norm = nn.LayerNorm(dim)
        self.net = nn.Sequential(
            nn.Linear(dim, dim * mult * 2),
            GEGLU(),
            nn.Dropout(dropout),
            nn.Linear(dim * mult, dim)
        )
        init_zero_(self.net[-1])

    def forward(self, x, **kwargs):
        x = self.norm(x)
        return self.net(x)

In [18]:
class Attention(nn.Module):
    def __init__(
            self,
            dim,
            heads,
            dim_head,
            dropout,
            gating_module=True
    ):
        super().__init__()
        inner_dim = dim_head * heads
        self.heads = heads
        self.scale = dim_head ** -0.5
        self.gating_module = gating_module
        self.to_q = nn.Linear(dim, inner_dim, bias=False)
        self.to_kv = nn.Linear(dim, inner_dim * 2, bias=False)
        self.to_out = nn.Linear(inner_dim, dim)
        if self.gating_module:
            self.gating = nn.Linear(dim, inner_dim)
            nn.init.constant_(self.gating.weight, 0.)
            nn.init.constant_(self.gating.bias, 1.)
        self.dropout = nn.Dropout(dropout)
        init_zero_(self.to_out)

    def forward(self, x, attn_bias=None, context=None, context_mask=None, tie_dim=None,output_attentions=False):
        device, orig_shape, h, has_context = x.device, x.shape, self.heads, exists(context)
        context = default(context, x)
        q, k, v = (self.to_q(x), *self.to_kv(context).chunk(2, dim=-1))
        q, k, v = map(lambda t: rearrange(t, 'b n (h d) -> b h n d', h=h), (q, k, v))
        q = q * self.scale

        if exists(tie_dim):
            q, k = map(lambda t: rearrange(t, '(b r) ... -> b r ...', r=tie_dim), (q, k))
            q = q.mean(dim=1)
            dots = einsum('b h i d, b r h j d -> b r h i j', q, k)
            dots = rearrange(dots, 'b r ... -> (b r) ...')
        else:
            dots = torch.einsum('b h i d, b h j d -> b h i j', q, k)

        if output_attentions:
            dots_out = dots
        if exists(attn_bias):
            dots = dots + attn_bias
        # attention
        dots = dots - dots.max(dim=-1, keepdims=True).values
        attn = dots.softmax(dim=-1)
        attn = self.dropout(attn)
        # aggregate
        out = torch.einsum('b h i j, b h j d -> b h i d', attn, v)
        # merge heads
        out = rearrange(out, 'b h n d -> b n (h d)')
        # gating
        if self.gating_module:
            gates = self.gating(x)
            out = out * gates.sigmoid()
        # combine to out
        out = self.to_out(out)
        if output_attentions:
            return out, attn, dots_out
        else:
            return out

In [19]:
class AxialAttention(nn.Module):
    def __init__(
            self,
            dim,
            heads,
            dropout=0.,
            row_attn=True,
            col_attn=True,
            accept_edges=False,
            global_query_attn=False,
            **kwargs
    ):
        super().__init__()
        assert not (not row_attn and not col_attn), 'row or column attention must be turned on'
        self.heads = heads
        self.row_attn = row_attn
        self.col_attn = col_attn
        self.global_query_attn = global_query_attn
        self.accept_edges = accept_edges

        self.norm = nn.LayerNorm(dim)
        self.attn = Attention(dim=dim, heads=heads,dropout=dropout, **kwargs)

    def forward(self, x, edges=None, output_attentions=False):
        assert self.row_attn ^ self.col_attn, 'has to be either row or column attention, but not both'

        b, h, w = x.shape

        # axial attention
        if self.row_attn:
            axial_dim = b
            input_fold_eq = 'b h w -> b w h'
            output_fold_eq = 'b h w -> b w h'
        elif self.col_attn:
            axial_dim = b
            input_fold_eq = 'b w h -> b w h'
            output_fold_eq = 'b w h -> b w h'

        x = rearrange(x, input_fold_eq)

        attn_bias = None
        if self.accept_edges and exists(edges):
            attn_bias = repeat(edges, 'b i j-> b x i j', x=self.heads)

        tie_dim = axial_dim if self.global_query_attn else None
        if output_attentions:
            out, attn_out_1, attn_out_2 = self.attn(x, attn_bias=attn_bias, tie_dim=tie_dim, output_attentions=output_attentions)
            out = rearrange(out, output_fold_eq)
            return out, attn_out_1, attn_out_2
        else:
            out = self.attn(x, attn_bias=attn_bias, tie_dim=tie_dim, output_attentions=output_attentions)
            out = rearrange(out, output_fold_eq)
            return out

In [20]:
class EvoformerBlock(nn.Module):
    def __init__(
            self,
            *,
            row_dim,
            col_dim,
            heads,
            dim_head,
            beta,
            attn_dropout=0.,
            ff_dropout=0.,
    ):
        super().__init__()
        self.beta=beta
        self.layer = nn.ModuleList([
            AxialAttention(dim=row_dim, heads=heads, dim_head=dim_head, dropout=attn_dropout, row_attn=True,col_attn=False),
            FeedForward(dim=row_dim, dropout=ff_dropout)
        ])

    def forward(self, m, output_attentions=False):
        msa_attn_row, msa_ff_row = self.layer
        # msa attention and transition
        if output_attentions:
            m_, attn_out_row_1, attn_out_row_2 = msa_attn_row(m, output_attentions=output_attentions)
            m = m_+m
            m = msa_ff_row(m.permute(0,2,1)) + m.permute(0,2,1)
            m = m.permute(0,2,1)
            m_, attn_out_col_1, attn_out_col_2 = msa_attn_col(m, output_attentions=output_attentions)
            m = self.beta*m_+m
            m = msa_ff_col(m) + m
        else:
            m = msa_attn_row(m) + m
            m = msa_ff_row(m.permute(0,2,1)) + m.permute(0,2,1)
            m = m.permute(0,2,1)

        # 更新x
        m=m.unsqueeze(3)
        if output_attentions:
            return m[:,:,:,0], attn_out_row_1, attn_out_row_2, attn_out_col_1, attn_out_col_2
        else:
            return  m[:,:,:,0]

In [21]:
class Evoformer(nn.Module):
    def __init__(
            self,
            depth,
            dim_network,
            row_dim,
            col_dim,
            heads,
            dim_head,
            embeding,
            beta,
            attn_dropout,
            ff_dropout,
            embeding_num,

    ):
        super().__init__()
        self.depth = depth
        self.dim_network = dim_network
        self.row_dim = row_dim
        self.col_dim = col_dim
        self.heads = heads
        self.dim_head = dim_head
        self.attn_dropout = attn_dropout
        self.ff_dropout = ff_dropout
        self.beta=beta
        self.embeding = embeding
        self.embeding_num=embeding_num
        if self.embeding:
            self.embedding_layer = Embedding_layer(feature_len=self.col_dim, embeding_dim=int(self.embeding_num/self.row_dim))
        self.layers = nn.ModuleList(
            [EvoformerBlock(row_dim=self.row_dim,col_dim=self.col_dim, heads=self.heads, dim_head=self.dim_head,beta=self.beta,
                              attn_dropout=self.attn_dropout, ff_dropout=self.ff_dropout) for _ in range(self.depth)])

    def forward(self, m, output_attentions=False):
        if self.embeding:
            m=self.embedding_layer(m.permute(0,2,1)).permute(0,2,1)
        attn_out_row_1_list = []
        attn_out_row_2_list = []
        attn_out_col_1_list = []
        attn_out_col_2_list = []
        for layer in self.layers:
            if output_attentions:
                m, attn_out_row_1, attn_out_row_2, attn_out_col_1, attn_out_col_2 = layer(m,output_attentions=output_attentions)
                attn_out_row_1_list.append(attn_out_row_1)
                attn_out_row_2_list.append(attn_out_row_2)
                attn_out_col_1_list.append(attn_out_col_1)
                attn_out_col_2_list.append(attn_out_col_2)
            else:
                m = layer(m)

        if output_attentions:
            return m,attn_out_row_1_list, attn_out_row_2_list, attn_out_col_1_list, attn_out_col_2_list
        else:
            return m

In [ ]:
class pathformer_model(nn.Module):
    def __init__(self,
                 depth,
                 row_dim,
                 col_dim,
                 heads,
                 dim_head,
                 classifier_input,  # 现在是一个数组，包含不同输入的特征数目
                 classifier_dim,
                 label_dim,
                 beta,
                 attn_dropout,
                 ff_dropout,
                 classifier_dropout,
                 num_view,
                 dim_hvcdn,
                 embeding=False,
                 embeding_num=32
                ):
        super(pathformer_model, self).__init__()
        self.Evoformer_model = Evoformer(depth=depth, dim_network=1, row_dim=row_dim,
                                         col_dim=col_dim, heads=heads, dim_head=dim_head,
                                         embeding=embeding, embeding_num=embeding_num,
                                         beta=beta, attn_dropout=attn_dropout, ff_dropout=ff_dropout)
        
        # 初始化 Graph_model 时不再使用单一的 classifier_input，而是使用不同输入的特征数目
        self.Graph_model_1 = Graph(classifier_input[0], classifier_dim[0], classifier_dropout, label_dim)  # 对应 Methylation
        self.Graph_model_2 = Graph(classifier_input[1], classifier_dim[1], classifier_dropout, label_dim)  # 对应 miRNASeq
        self.Graph_model_3 = Graph(classifier_input[2], classifier_dim[2], classifier_dropout, label_dim)  # 对应 RNAseq
        
#         self.VCDN = VCDN(num_view+1, label_dim, dim_hvcdn)
#         self.HFBSurv = HFBSurv(classifier_dim[0][-1], dim_hvcdn, label_dim, attn_dropout, 20, classifier_dropout)
        self.CMFM = CMFM(classifier_dim[0][-1], label_dim, classifier_dropout, dim_hvcdn)
        self.TCP = TCP(classifier_input, [classifier_dim[0][-1]], label_dim, classifier_dropout)

    def forward(self, Methylation, miRNASeq, RNAseq, graph, label, output_attentions):
        # 获取 Evoformer 模型的输出
        if output_attentions:
            m, attn_out_row_1_list, attn_out_row_2_list, attn_out_col_1_list, attn_out_col_2_list = self.Evoformer_model(graph, output_attentions=output_attentions)
        else:
            m = self.Evoformer_model(graph, output_attentions=output_attentions)
        
        # 拆分 m
        m_splits = torch.unbind(m, dim=1)
#         # 拆分 m
#         m_splits = torch.unbind(graph, dim=1)

        # 将每个切分后的部分分别传入对应的 Graph_model
        dec_logits1 = self.Graph_model_1(Methylation, m_splits[0])  # Methylation 对应的 Graph_model
        dec_logits2 = self.Graph_model_2(miRNASeq, m_splits[1])         # miRNASeq 对应的 Graph_model
        dec_logits3 = self.Graph_model_3(RNAseq, m_splits[2])       # RNAseq 对应的 Graph_model
        # 将四个结果合成列表
#         result_list = [dec_logits1, dec_logits2, dec_logits3]
        
        # 将结果传入 VCDN 模块生成最终输出
#         output = self.VCDN(result_list)
#         output = self.VCDN(dec_logits1, dec_logits2, dec_logits3)
#         output = self.HFBSurv(dec_logits1, dec_logits2, dec_logits3)
        output, _, _, _ = self.CMFM(dec_logits1, dec_logits2, dec_logits3)
#       将四个结果合成列表
        result_list = [output, dec_logits1, dec_logits2, dec_logits3]
#         output = self.VCDN(result_list)
        label = label.squeeze()
        loss, output, feature = self.TCP(result_list, label)
    
        if output_attentions:
            return output, attn_out_row_1_list, attn_out_row_2_list, attn_out_col_1_list, attn_out_col_2_list, m
        else:
            return loss, output, feature